In [1]:
import numpy as np
import pandas as pd
from utm import from_latlon

In [313]:
actype_df = pd.read_csv('aircraft_type.csv')
# df_list = [i[1] for i in df.groupby(['callsign', 'icao24'])]
actype_df.columns = ['Manufacturer', 'Model', 'Type Designator', 'Description', 'Engine Type', 'Engine Count', 'WTC']
actype_wtc_dict = dict(zip(actype_df['Type Designator'], actype_df['WTC']))
wtc_to_int_dict = {'M': 0, 'L': 1, 'H': 2, 'L/M': 3, 'J': 4}
north_east = lambda x: from_latlon(x['latitude'],x['longitude'])

def convert_delta(g1):
    g1.groupby(['unique_id','zone', 'band'], as_index=False).nth(0).index[1:]
    # Get index of first row of each zone (except first zone)
    origin_index = g1.groupby(['unique_id','zone', 'band'], as_index=False).nth(0).index[1:]
    # Check if group only has 1 unique zone
    if len(origin_index) == 0:
        g1[['final_delta_e', 'final_delta_n']] = g1[['delta_e', 'delta_n']]
        return g1
    # Calculate inverse of first row of each zone in latitude/longitude
    inverse = g1.loc[origin_index - 1].reset_index()[['latitude','longitude']] - (g1.loc[origin_index].reset_index()[['latitude','longitude']] - g1.loc[origin_index - 1].reset_index()[['latitude','longitude']])
    # Check if inverse is within bounds of longitude/latitude
    if len(inverse[(inverse['latitude'] > 84) | (inverse['latitude'] < -80) | (inverse['longitude'] > 180) | (inverse['longitude'] < -180)]) != 0:
        return
    # Convert inverse to UTM
    inverse_results = inverse.apply(north_east, axis=1, result_type='expand')
    # Check if inverse is within same zone as the last row of the previous zone
    if 0 in (inverse_results[[2,3]].values == g1.loc[origin_index-1][['zone', 'band']].values):
        return
    # Calculate new origin estimate in UTM
    new_origin = g1.loc[origin_index-1][['easting', 'northing']].values - inverse_results[[0,1]].values + g1.loc[origin_index-1][['easting', 'northing']].values
    # Calculate new origin estimate in UTM delta with respect to the first row of the previous zone
    g1.loc[origin_index, ['origin_e', 'origin_n']] = new_origin - g1.loc[origin_index-1][['easting', 'northing']].values + g1.loc[origin_index-1][['delta_e', 'delta_n']].values
    # Calculate final delta in UTM for every row in the group
    g1['delta_cum_n'] = g1['origin_n'].cumsum()
    g1['delta_cum_e'] = g1['origin_e'].cumsum()
    g1['final_delta_n'] = g1['delta_n'] + g1['delta_cum_n']
    g1['final_delta_e'] = g1['delta_e'] + g1['delta_cum_e']
    g1.drop(['delta_cum_n', 'delta_cum_e'], axis=1, inplace=True)

    return g1

In [383]:
df = pd.read_csv('traffic_22.csv')
initial_length = len(df)
df['wtc'] = df['actype'].map(actype_wtc_dict)
df.dropna(inplace=True)
df['label'] = df['wtc'].map(wtc_to_int_dict).astype(int)

df['last_position'] = pd.to_datetime(df['last_position'])
df['filter'] = df.groupby(['callsign', 'icao24'])['last_position'].diff() > pd.Timedelta(2,'s')
df['count'] = df.groupby(['callsign', 'icao24'])['filter'].transform(pd.Series.cumsum).astype(str)
# df = df.groupby(['callsign', 'icao24', 'count']).filter(lambda group: len(group) >= 200)
df['subgroup_id'] = (df.groupby(['callsign', 'icao24', 'count']).cumcount() // 200).astype(str)
df['long'] = df.groupby(['callsign', 'icao24', 'count', 'subgroup_id'])['longitude'].transform('first').astype(str).str[-5:]
df['unique_id'] = df['callsign'] + '_' + df['icao24'] + '_' + df['count'] + '_' + df['long'] + '_' + df['subgroup_id']
df = df.groupby('unique_id').filter(lambda group: len(group) == 200)
# df.drop(['filter', 'count', 'long', ''], axis=1, inplace=True)

df[['easting', 'northing', 'zone', 'band']] = df[['latitude', 'longitude']].apply(north_east, axis=1, result_type='expand')
df[['origin_e', 'origin_n']] = 0
df['origin_a'] = df.groupby('unique_id')['altitude'].transform('first')
df['delta_a'] = df['altitude'] - df['origin_a']
df['origin_ga'] = df.groupby('unique_id')['geoaltitude'].transform('first')
df['delta_ga'] = df['geoaltitude'] - df['origin_ga']
df[['delta_e', 'delta_n']] = df[['easting', 'northing']] - df.groupby(['unique_id','zone','band'])[['easting', 'northing']].transform('first')
df = df.groupby(['unique_id'], group_keys=False).apply(lambda x: convert_delta(x)).drop(['origin_e', 'origin_n', 'delta_e', 'delta_n'], axis=1)

# df.dropna(inplace=True)
# df['label'] = df['label'].astype(int)
df = df[['final_delta_n', 'final_delta_e', 'delta_a', 'label', 'unique_id']]

In [312]:
actype_df['WTC'].value_counts(dropna=False)

L      7935
M      2168
H       214
L/M      96
J         3
Name: WTC, dtype: int64

In [315]:
len(df)

22000

In [385]:
df.shape

(22000, 5)

In [387]:
wrong = pd.read_csv('output.csv')

In [388]:
wrong

,final_delta_n,final_delta_e,delta_a,label,unique_id
0,0.000000,0.000000,0.0,0,AC402_c0671b_0_63842_0
1,77.517985,127.466269,-25.0,0,AC402_c0671b_0_63842_0
2,180.286093,298.464578,-50.0,0,AC402_c0671b_0_63842_0
3,268.706991,438.451197,-75.0,0,AC402_c0671b_0_63842_0
4,374.695934,611.784618,-100.0,0,AC402_c0671b_0_63842_0
...,...,...,...,...,...
381195,-7371.742464,-25705.814509,-4000.0,0,ACA133_c05eb1_0_85177_0
381196,-7413.371924,-25835.081473,-4025.0,0,ACA133_c05eb1_0_85177_0
381197,-7448.887286,-25965.981845,-4025.0,0,ACA133_c05eb1_0_85177_0
381198,-7495.174421,-26130.798070,-4050.0,0,ACA133_c05eb1_0_85177_0


In [280]:
df.dropna(inplace=True)

In [310]:
df.wtc.value_counts(dropna=False)

M      32170
L       6352
L/M     1667
H        480
Name: wtc, dtype: int64

In [284]:
for i in df.wtc.unique():
    if i not in wtc_to_int_dict.keys():
        print(i)

0.0
1.0
nan


In [395]:
a = wrong.groupby(['unique_id'], sort=False).apply(np.array).tolist()

In [397]:
for i in a:
    if i.shape != (200, 5):
        print(i.shape)

(400, 5)
(400, 5)


In [407]:
bruh = wrong.groupby(['unique_id'], sort=False).apply(lambda x: x if len(x) == 200 else None).dropna()

In [409]:
bruh

final_delta_n  final_delta_e  delta_a  label  \
unique_id                                                                      
AC402_c0671b_0_63842_0  0            0.000000       0.000000      0.0      0   
                        1           77.517985     127.466269    -25.0      0   
                        2          180.286093     298.464578    -50.0      0   
                        3          268.706991     438.451197    -75.0      0   
                        4          374.695934     611.784618   -100.0      0   
...                                       ...            ...      ...    ...   
ACA133_c05eb1_0_85177_0 381195   -7371.742464  -25705.814509  -4000.0      0   
                        381196   -7413.371924  -25835.081473  -4025.0      0   
                        381197   -7448.887286  -25965.981845  -4025.0      0   
                        381198   -7495.174421  -26130.798070  -4050.0      0   
                        381199   -7516.382030  -26196.312855  -4075.0      0   

                                              unique_id  
unique_id                                                
AC402_c0671b_0_63842_0  0        AC402_c0671b_0_63842_0  
                        1        AC402_c0671b_0_63842_0  
                        2        AC402_c0671b_0_63842_0  
                        3        AC402_c0671b_0_63842_0  
                        4        AC402_c0671b_0_63842_0  
...                                                 ...  
ACA133_c05eb1_0_85177_0 381195  ACA133_c05eb1_0_85177_0  
                        381196  ACA133_c05eb1_0_85177_0  
                        381197  ACA133_c05eb1_0_85177_0  
                        381198  ACA133_c05eb1_0_85177_0  
                        381199  ACA133_c05eb1_0_85177_0  

[380400 rows x 5 columns]

In [410]:
arr = np.array(wrong.groupby(['unique_id'], sort=False).apply(lambda x: np.array(x) if len(x)==200 else None).dropna().tolist())[:, :, :-1]
label_arr = arr[:, :, -1][:, 0].astype(int)
arr = arr[:, :, :-1].astype(np.float32)

In [411]:
label_arr.shape

(1902,)

In [412]:
arr.shape

(1902, 200, 3)

In [375]:
from torch.utils.data import Dataset, DataLoader
from torch import save, load

In [363]:
class TrajectoryDataset(Dataset):
    def __init__(self, data_arr, label_arr, transform=None, target_transform=None):
        self.data_arr = data_arr
        self.label_arr = label_arr
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.label_arr)

    def __getitem__(self, idx):
        trajectory = self.data_arr[idx]
        label = self.label_arr[idx]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return trajectory, label

In [382]:
df.shape

(22000, 5)

In [376]:
save(TrajectoryDataset(arr, label_arr), 'traffic_22.pt')

In [364]:
data = TrajectoryDataset(arr, label_arr)

In [446]:
train_dataloader = DataLoader(load('dataset.pt'), batch_size=32, shuffle=True)

In [447]:
train_dataloader.dataset.data_arr.shape

(1239, 200, 3)

In [448]:
train_features, train_labels = next(iter(train_dataloader))

In [452]:
train_features

tensor([[[ 0.0000e+00,  0.0000e+00,  0.0000e+00],
         [-5.0878e+01, -8.5768e+01, -2.5000e+01],
         [-1.0026e+02, -1.6526e+02, -2.5000e+01],
         ...,
         [ 5.0513e+03, -9.1894e+03, -2.0000e+03],
         [ 5.1166e+03, -9.1117e+03, -2.0000e+03],
         [ 5.1754e+03, -9.0411e+03, -2.0000e+03]],

        [[ 0.0000e+00,  0.0000e+00,  0.0000e+00],
         [-1.0337e+02, -9.0167e+01,  0.0000e+00],
         [-2.0333e+02, -1.8289e+02,  0.0000e+00],
         ...,
         [-1.9904e+04, -1.7796e+04, -2.7750e+03],
         [-1.9998e+04, -1.7879e+04, -2.8000e+03],
         [-2.0109e+04, -1.7977e+04, -2.8000e+03]],

        [[ 0.0000e+00,  0.0000e+00,  0.0000e+00],
         [ 1.0073e+02, -1.2137e+02,  0.0000e+00],
         [ 1.5497e+02, -1.8553e+02,  0.0000e+00],
         ...,
         [ 1.7326e+04, -2.1865e+04, -1.4000e+03],
         [ 1.7367e+04, -2.2007e+04, -1.4250e+03],
         [ 1.7403e+04, -2.2136e+04, -1.4750e+03]],

        ...,

        [[ 0.0000e+00,  0.0000e+00,  0

In [450]:
train_labels.shape

torch.Size([32])

In [451]:
train_labels

tensor([0, 2, 2, 0, 0, 0, 0, 0, 0, 2, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 1])

In [324]:
arr.dtype

dtype('float64')

In [319]:
arr.shape

(110, 200, 4)

In [197]:
df = pd.read_csv('traffic_22.csv')
initial_length = len(df)
df['last_position'] = pd.to_datetime(df['last_position'])
df['filter'] = df.groupby(['callsign', 'icao24'])['last_position'].diff() > pd.Timedelta(2,'s')
df['count'] = df.groupby(['callsign', 'icao24'])['filter'].transform(pd.Series.cumsum).astype(str)
df['long'] = df.groupby(['callsign', 'icao24', 'count'])['longitude'].transform('first').astype(str).str[-5:]
df['unique_id'] = df['callsign'] + '_' + df['icao24'] + '_' + df['count'] + '_' + df['long']
df = df.groupby(['unique_id']).filter(lambda group: len(group) >= 200)
df.drop(['filter', 'count', 'long'], axis=1, inplace=True)
df['subgroup_id'] = df.groupby('unique_id').cumcount() // 200
df['unique_subgroup_id'] = df.groupby(['unique_id', 'subgroup_id']).ngroup()
df = df.groupby(['unique_subgroup_id']).filter(lambda group: len(group) >= 200)

In [200]:
len(df.groupby(['unique_id']))

102

In [179]:
df.to_csv('traffic_22_test.csv', index=False)

In [161]:
df['subgroup_id'] = df.groupby('unique_id').cumcount() // 200

In [162]:
df['unique_subgroup_id'] = df.groupby(['unique_id', 'subgroup_id']).ngroup()

In [145]:
pd.options.display.max_rows = 500

In [191]:
len(df.groupby(['unique_id']))

115

In [183]:
df

,altitude,callsign,geoaltitude,groundspeed,icao24,last_position,latitude,longitude,onground,spi,squawk,timestamp,track,vertical_rate,actype,subgroup_id,unique_id
1268,4825.0,ANT537,5100.0,247.0,c0460f,2022-01-14 21:44:37.898999808+00:00,48.671814,-123.271883,False,False,7003.0,2022-01-14 21:44:38+00:00,329.910314,1536.0,B735,0,ANT537_c0460f_1_10217_0
1269,4850.0,ANT537,5125.0,249.0,c0460f,2022-01-14 21:44:38.440999936+00:00,48.672592,-123.272588,False,False,7003.0,2022-01-14 21:44:39+00:00,328.483060,1536.0,B735,0,ANT537_c0460f_1_10217_0
1270,4875.0,ANT537,5150.0,250.0,c0460f,2022-01-14 21:44:39.595000064+00:00,48.673670,-123.273613,False,False,7003.0,2022-01-14 21:44:40+00:00,327.264774,1536.0,B735,0,ANT537_c0460f_1_10217_0
1271,4900.0,ANT537,5175.0,251.0,c0460f,2022-01-14 21:44:40.815000064+00:00,48.674834,-123.274841,False,False,7003.0,2022-01-14 21:44:41+00:00,326.373243,1472.0,B735,0,ANT537_c0460f_1_10217_0
1272,4925.0,ANT537,5200.0,252.0,c0460f,2022-01-14 21:44:41.811000064+00:00,48.675765,-123.275853,False,False,7003.0,2022-01-14 21:44:42+00:00,325.175511,1472.0,B735,0,ANT537_c0460f_1_10217_0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41899,7775.0,WSW210,7925.0,223.0,c028cd,2022-01-11 03:49:29.651000064+00:00,48.787720,-123.404564,False,False,1341.0,2022-01-11 03:49:30+00:00,220.462227,-1408.0,B738,0,WSW210_c028cd_0_74589_0
41900,7775.0,WSW210,7925.0,223.0,c028cd,2022-01-11 03:49:30.663000064+00:00,48.786850,-123.405621,False,False,1341.0,2022-01-11 03:49:31+00:00,220.266562,-1344.0,B738,0,WSW210_c028cd_0_74589_0
41901,7750.0,WSW210,7900.0,223.0,c028cd,2022-01-11 03:49:31.532000+00:00,48.786163,-123.406536,False,False,1341.0,2022-01-11 03:49:32+00:00,220.266562,-1344.0,B738,0,WSW210_c028cd_0_74589_0
41902,7725.0,WSW210,7875.0,223.0,c028cd,2022-01-11 03:49:32.838000128+00:00,48.785256,-123.407689,False,False,1341.0,2022-01-11 03:49:33+00:00,220.629203,-1280.0,B738,0,WSW210_c028cd_0_74589_0


In [164]:
df = df.groupby('unique_subgroup_id').filter(lambda group: len(group) >= 200)

In [165]:
len(df.groupby('unique_subgroup_id'))

115

In [32]:
df = pd.read_csv('traffic_22.csv')
initial_length = len(df)
df['last_position'] = pd.to_datetime(df['last_position'])

In [76]:
df['filter'] = False

In [133]:
df['filter'] = df.groupby(['callsign', 'icao24']).transform(lambda x: (x.index % 200 == 0) & (x.index != 0)).astype(int).max(axis=1)

In [134]:
df['filter'] = ((df.groupby(['callsign', 'icao24'])['last_position'].diff() > pd.Timedelta(2,'s')) | (df['filter'] == 1)).astype(int)

In [136]:
asdf.value_counts()

0    222
1      2
Name: filter, dtype: int64

In [139]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(asdf)

     altitude callsign  geoaltitude  groundspeed  icao24  \
0      9425.0   ACA033       9375.0        259.0  c01732   
1      9500.0   ACA033       9475.0        259.0  c01732   
2      9550.0   ACA033       9525.0        258.0  c01732   
3      9625.0   ACA033       9575.0        258.0  c01732   
4      9650.0   ACA033       9625.0        258.0  c01732   
5      9700.0   ACA033       9675.0        258.0  c01732   
6      9750.0   ACA033       9725.0        258.0  c01732   
7      9825.0   ACA033       9750.0        258.0  c01732   
8      9875.0   ACA033       9850.0        257.0  c01732   
9      9925.0   ACA033       9900.0        257.0  c01732   
10     9975.0   ACA033       9950.0        257.0  c01732   
11    10025.0   ACA033      10000.0        256.0  c01732   
12    10075.0   ACA033      10050.0        256.0  c01732   
13    10150.0   ACA033      10100.0        255.0  c01732   
14    10200.0   ACA033      10175.0        255.0  c01732   
15    10250.0   ACA033      10225.0     

In [138]:
asdf = df.groupby(['callsign', 'icao24']).get_group(('ACA033', 'c01732'))

In [78]:
df.loc[(df.groupby(['callsign', 'icao24'])['last_position'].diff() > pd.Timedelta(2,'s')), 'filter'] = True

In [81]:
df.loc[(df.groupby(['callsign', 'icao24']).apply(lambda x: x.index % 200 == 0)), 'filter'] = True

TypeError: unhashable type: 'numpy.ndarray'

In [73]:
unique_group.apply(lambda x: x.index % 200 == 0)

callsign  icao24
ACA033    c01732    [True, False, False, False, False, False, Fals...
ACA1054   c05030    [False, False, False, False, False, False, Fal...
          c05054    [False, False, False, False, False, False, Fal...
ACA1060   c04fff    [False, False, False, False, False, False, Fal...
          c05030    [False, False, False, False, False, False, Fal...
                                          ...                        
WSW189    c01d3c                                              [False]
          c08335    [False, False, False, False, False, False, Fal...
WSW190    c01d3c    [False, False, False, False, False, False, Fal...
          c08335                                              [False]
WSW210    c028cd    [False, False, False, False, False, False, Fal...
Length: 161, dtype: object

In [65]:
unique_group = df.groupby(['callsign', 'icao24'])
df['filter'] = (unique_group['last_position'].diff() > pd.Timedelta(2,'s') | (unique_group.apply(lambda x: x.index % 200 == 0)))
df

TypeError: Cannot perform 'ror_' with a dtyped [object] array and scalar of type [bool]

In [12]:
idx = np.arange(len(df.index))//200

In [16]:
df.groupby(idx)

In [22]:
len(df.groupby('unique_id'))

102

In [26]:
g1 = df.groupby('unique_id').get_group('ANT537_c0460f_7_10217')

In [27]:
g1

,altitude,callsign,geoaltitude,groundspeed,icao24,last_position,latitude,longitude,onground,spi,squawk,timestamp,track,vertical_rate,actype,unique_id
1268,4825.0,ANT537,5100.0,247.0,c0460f,2022-01-14 21:44:37.898999808+00:00,48.671814,-123.271883,False,False,7003.0,2022-01-14 21:44:38+00:00,329.910314,1536.0,B735,ANT537_c0460f_7_10217
1269,4850.0,ANT537,5125.0,249.0,c0460f,2022-01-14 21:44:38.440999936+00:00,48.672592,-123.272588,False,False,7003.0,2022-01-14 21:44:39+00:00,328.483060,1536.0,B735,ANT537_c0460f_7_10217
1270,4875.0,ANT537,5150.0,250.0,c0460f,2022-01-14 21:44:39.595000064+00:00,48.673670,-123.273613,False,False,7003.0,2022-01-14 21:44:40+00:00,327.264774,1536.0,B735,ANT537_c0460f_7_10217
1271,4900.0,ANT537,5175.0,251.0,c0460f,2022-01-14 21:44:40.815000064+00:00,48.674834,-123.274841,False,False,7003.0,2022-01-14 21:44:41+00:00,326.373243,1472.0,B735,ANT537_c0460f_7_10217
1272,4925.0,ANT537,5200.0,252.0,c0460f,2022-01-14 21:44:41.811000064+00:00,48.675765,-123.275853,False,False,7003.0,2022-01-14 21:44:42+00:00,325.175511,1472.0,B735,ANT537_c0460f_7_10217
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1522,14825.0,ANT537,15050.0,369.0,c0460f,2022-01-14 21:48:51.992999936+00:00,48.919693,-123.689223,False,False,7003.0,2022-01-14 21:48:52+00:00,300.245268,3328.0,B735,ANT537_c0460f_7_10217
1523,14850.0,ANT537,15075.0,369.0,c0460f,2022-01-14 21:48:52.499000064+00:00,48.920197,-123.690561,False,False,7003.0,2022-01-14 21:48:53+00:00,300.245268,3392.0,B735,ANT537_c0460f_7_10217
1524,14900.0,ANT537,15150.0,369.0,c0460f,2022-01-14 21:48:53.519000064+00:00,48.921066,-123.692815,False,False,7003.0,2022-01-14 21:48:54+00:00,300.245268,3264.0,B735,ANT537_c0460f_7_10217
1525,14975.0,ANT537,15250.0,369.0,c0460f,2022-01-14 21:48:54.934000128+00:00,48.922302,-123.695913,False,False,7003.0,2022-01-14 21:48:55+00:00,300.245268,3136.0,B735,ANT537_c0460f_7_10217


In [29]:
len(g1.groupby(np.arange(len(g1.index))//10))

26

In [23]:
df.groupby(['unique_id']).apply(lambda x: x.groupby(np.arange(len(x.index))//10))

unique_id
ANT537_c0460f_7_10217       <pandas.core.groupby.generic.DataFrameGroupBy ...
ASP503_c001eb_7_10217       <pandas.core.groupby.generic.DataFrameGroupBy ...
CFBCL_c002e4_11_33717       <pandas.core.groupby.generic.DataFrameGroupBy ...
CFMJO_c020a9_14_58534       <pandas.core.groupby.generic.DataFrameGroupBy ...
CGHJK_c05819_16_93051       <pandas.core.groupby.generic.DataFrameGroupBy ...
                                                  ...                        
WJA2041_c0226c_209_28606    <pandas.core.groupby.generic.DataFrameGroupBy ...
WJA995_c0226c_215_09615     <pandas.core.groupby.generic.DataFrameGroupBy ...
WSW189_c08335_216_98314     <pandas.core.groupby.generic.DataFrameGroupBy ...
WSW190_c01d3c_217_60979     <pandas.core.groupby.generic.DataFrameGroupBy ...
WSW210_c028cd_218_59615     <pandas.core.groupby.generic.DataFrameGroupBy ...
Length: 102, dtype: object

In [251]:
df = pd.read_csv('aircraft_type.csv')

In [217]:
df = pd.read_csv('out_p200s2.csv')
df

,Unnamed: 0,altitude,callsign,geoaltitude,groundspeed,icao24,last_position,latitude,longitude,onground,spi,squawk,timestamp,track,vertical_rate,actype,unique_id
0,1268,4825.0,ANT537,5100.0,247.0,c0460f,2022-01-14 21:44:37.898999808+00:00,48.671814,-123.271883,False,False,7003.0,2022-01-14 21:44:38+00:00,329.910314,1536.0,B735,ANT537_c0460f_1_10217
1,1269,4850.0,ANT537,5125.0,249.0,c0460f,2022-01-14 21:44:38.440999936+00:00,48.672592,-123.272588,False,False,7003.0,2022-01-14 21:44:39+00:00,328.483060,1536.0,B735,ANT537_c0460f_1_10217
2,1270,4875.0,ANT537,5150.0,250.0,c0460f,2022-01-14 21:44:39.595000064+00:00,48.673670,-123.273613,False,False,7003.0,2022-01-14 21:44:40+00:00,327.264774,1536.0,B735,ANT537_c0460f_1_10217
3,1271,4900.0,ANT537,5175.0,251.0,c0460f,2022-01-14 21:44:40.815000064+00:00,48.674834,-123.274841,False,False,7003.0,2022-01-14 21:44:41+00:00,326.373243,1472.0,B735,ANT537_c0460f_1_10217
4,1272,4925.0,ANT537,5200.0,252.0,c0460f,2022-01-14 21:44:41.811000064+00:00,48.675765,-123.275853,False,False,7003.0,2022-01-14 21:44:42+00:00,325.175511,1472.0,B735,ANT537_c0460f_1_10217
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30548,42045,5550.0,WSW210,5700.0,218.0,c028cd,2022-01-11 03:51:55.795000064+00:00,48.773571,-123.598577,False,False,1341.0,2022-01-11 03:51:56+00:00,284.355259,-1216.0,B738,WSW210_c028cd_0_74589
30549,42046,5525.0,WSW210,5700.0,218.0,c028cd,2022-01-11 03:51:56.885999872+00:00,48.773804,-123.600094,False,False,1341.0,2022-01-11 03:51:57+00:00,283.014354,-1216.0,B738,WSW210_c028cd_0_74589
30550,42047,5500.0,WSW210,5650.0,216.0,c028cd,2022-01-11 03:51:57.956000+00:00,48.773987,-123.601614,False,False,1341.0,2022-01-11 03:51:58+00:00,281.465768,-1152.0,B738,WSW210_c028cd_0_74589
30551,42048,5475.0,WSW210,5650.0,215.0,c028cd,2022-01-11 03:51:58.960999936+00:00,48.774170,-123.603234,False,False,1341.0,2022-01-11 03:51:59+00:00,279.637538,-1088.0,B738,WSW210_c028cd_0_74589


In [18]:
north_east = lambda x: from_latlon(x['latitude'],x['longitude'])

In [221]:
df[['easting', 'northing', 'zone', 'band']] = df[['latitude', 'longitude']].apply(north_east, axis=1, result_type='expand')
df[['origin_e', 'origin_n']] = 0
df[['delta_e', 'delta_n']] = df[['easting', 'northing']] - df.groupby(['unique_id','zone','band'])[['easting', 'northing']].transform('first')

In [2]:
df.head()

NameError: name 'df' is not defined

In [106]:
list(from_latlon(30, 140))

[403549.84743286244, 3319206.222755263, 54, 'R']

In [13]:
df.groupby(['zone', 'band'])

In [236]:
df

,Unnamed: 0,altitude,callsign,geoaltitude,groundspeed,icao24,last_position,latitude,longitude,onground,...,actype,unique_id,easting,northing,zone,band,origin_e,origin_n,delta_e,delta_n
0,1268,4825.0,ANT537,5100.0,247.0,c0460f,2022-01-14 21:44:37.898999808+00:00,48.671814,-123.271883,False,...,B735,ANT537_c0460f_1_10217,479983.441269,5.391010e+06,10,U,0,0,0.000000,0.000000
1,1269,4850.0,ANT537,5125.0,249.0,c0460f,2022-01-14 21:44:38.440999936+00:00,48.672592,-123.272588,False,...,B735,ANT537_c0460f_1_10217,479931.902058,5.391096e+06,10,U,0,0,-51.539211,86.688534
2,1270,4875.0,ANT537,5150.0,250.0,c0460f,2022-01-14 21:44:39.595000064+00:00,48.673670,-123.273613,False,...,B735,ANT537_c0460f_1_10217,479856.879958,5.391216e+06,10,U,0,0,-126.561311,206.752657
3,1271,4900.0,ANT537,5175.0,251.0,c0460f,2022-01-14 21:44:40.815000064+00:00,48.674834,-123.274841,False,...,B735,ANT537_c0460f_1_10217,479766.887735,5.391346e+06,10,U,0,0,-216.553534,336.445034
4,1272,4925.0,ANT537,5200.0,252.0,c0460f,2022-01-14 21:44:41.811000064+00:00,48.675765,-123.275853,False,...,B735,ANT537_c0460f_1_10217,479692.768623,5.391450e+06,10,U,0,0,-290.672647,440.207697
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30548,42045,5550.0,WSW210,5700.0,218.0,c028cd,2022-01-11 03:51:55.795000064+00:00,48.773571,-123.598577,False,...,B738,WSW210_c028cd_0_74589,456020.583608,5.402458e+06,10,U,0,0,-27780.070948,-21978.595352
30549,42046,5525.0,WSW210,5700.0,218.0,c028cd,2022-01-11 03:51:56.885999872+00:00,48.773804,-123.600094,False,...,B738,WSW210_c028cd_0_74589,455909.266615,5.402485e+06,10,U,0,0,-27891.387940,-21951.844193
30550,42047,5500.0,WSW210,5650.0,216.0,c028cd,2022-01-11 03:51:57.956000+00:00,48.773987,-123.601614,False,...,B738,WSW210_c028cd_0_74589,455797.770900,5.402506e+06,10,U,0,0,-28002.883655,-21930.609472
30551,42048,5475.0,WSW210,5650.0,215.0,c028cd,2022-01-11 03:51:58.960999936+00:00,48.774170,-123.603234,False,...,B738,WSW210_c028cd_0_74589,455678.923069,5.402527e+06,10,U,0,0,-28121.731486,-21909.314301


In [107]:
test = pd.read_csv('utm_test2.csv', delimiter=',')
test[['easting', 'northing', 'zone', 'band']] = test[['latitude', 'longitude']].apply(north_east, axis=1, result_type='expand')
test[['origin_e', 'origin_n']] = 0
test[['delta_e', 'delta_n']] = test[['easting', 'northing']] - test.groupby(['unique_id','zone','band'])[['easting', 'northing']].transform('first')

In [240]:
out = test.groupby(['unique_id'], group_keys=False).apply(lambda x: convert_delta(x)).drop(['origin_e', 'origin_n', 'delta_e', 'delta_n'], axis=1)
out

,latitude,longitude,unique_id,easting,northing,zone,band,final_delta_n,final_delta_e
0,80,176,1,480615.196704,8.881752e+06,60,X,0.000000e+00,0.000000
1,80,175,1,461235.942285,8.882252e+06,60,X,4.997549e+02,-19379.254419
2,80,174,1,441867.784868,8.883085e+06,60,X,1.332541e+03,-38747.411836
3,80,173,1,538764.057715,8.882252e+06,59,X,2.165327e+03,-58115.569253
4,80,172,1,519384.803296,8.881752e+06,59,X,1.665572e+03,-77494.823672
5,80,171,1,500000.000000,8.881586e+06,59,X,1.498974e+03,-96879.626968
6,80,170,1,480615.196704,8.881752e+06,59,X,1.665572e+03,-116264.430264
7,73,170,1,467367.675522,8.100752e+06,59,X,-7.793347e+05,-129511.951446
8,72,170,1,465510.981461,7.989219e+06,59,X,-8.908681e+05,-131368.645507
9,71,170,1,463664.972582,7.877697e+06,59,W,-1.002401e+06,-133225.339568


In [237]:
def convert_delta(g1):
    g1.groupby(['unique_id','zone', 'band'], as_index=False).nth(0).index[1:]
    # Get index of first row of each zone (except first zone)
    origin_index = g1.groupby(['unique_id','zone', 'band'], as_index=False).nth(0).index[1:]
    # Check if group only has 1 unique zone
    if len(origin_index) == 0:
        g1[['final_delta_e', 'final_delta_n']] = g1[['delta_e', 'delta_n']]
        return g1
    # Calculate inverse of first row of each zone in latitude/longitude
    inverse = g1.loc[origin_index - 1].reset_index()[['latitude','longitude']] - (g1.loc[origin_index].reset_index()[['latitude','longitude']] - g1.loc[origin_index - 1].reset_index()[['latitude','longitude']])
    # Check if inverse is within bounds of longitude/latitude
    if len(inverse[(inverse['latitude'] > 84) | (inverse['latitude'] < -80) | (inverse['longitude'] > 180) | (inverse['longitude'] < -180)]) != 0:
        return
    # Convert inverse to UTM
    inverse_results = inverse.apply(north_east, axis=1, result_type='expand')
    # Check if inverse is within same zone as the last row of the previous zone
    if 0 in (inverse_results[[2,3]].values == g1.loc[origin_index-1][['zone', 'band']].values):
        return
    # Calculate new origin estimate in UTM
    new_origin = g1.loc[origin_index-1][['easting', 'northing']].values - inverse_results[[0,1]].values + g1.loc[origin_index-1][['easting', 'northing']].values
    # Calculate new origin estimate in UTM delta with respect to the first row of the previous zone
    g1.loc[origin_index, ['origin_e', 'origin_n']] = new_origin - g1.loc[origin_index-1][['easting', 'northing']].values + g1.loc[origin_index-1][['delta_e', 'delta_n']].values
    # Calculate final delta in UTM for every row in the group
    g1['delta_cum_n'] = g1['origin_n'].cumsum()
    g1['delta_cum_e'] = g1['origin_e'].cumsum()
    g1['final_delta_n'] = g1['delta_n'] + g1['delta_cum_n']
    g1['final_delta_e'] = g1['delta_e'] + g1['delta_cum_e']
    g1.drop(['delta_cum_n', 'delta_cum_e'], axis=1, inplace=True)

    return g1

In [ ]:
for _, g1 in test.groupby(['unique_id']):
    # Check if group only has 1 unique zone
    origin_index = g1.groupby(['unique_id','zone', 'band'], as_index=False).nth(0).index[1:]

In [448]:
df.groupby('unique_id').get_group('ANT537_c0460f_1_10217').groupby(['unique_id','zone', 'band'], as_index=False).nth(0).index[1:]

Int64Index([], dtype='int64')

In [ ]:
# Test case where 2 zone/band jumps in a row (i.e. 1 jump from zone 32 to 33 and band R to R, then another jump from zone 33 to 33 and band R to S) (realistically not possible but should be valid and left as is)
# Test case where if only 1 unique zone+band per group (i.e. 1 jump from zone 32 to 33 and band R to R, or 1 jump from zone 32 to 32 and band R to S) then leave as is
# Test case to make sure if jump both zone and band at the same time (i.e. 1 jump from zone 32 to 33 and band R to S) then discard group
# Test case where 2 zone/band changes in 1 jump (i.e. 1 jump from zone 32 to 35 OR band R to T) (realistically not possible and will break) (then discard group?)

In [442]:
origin_index = df.groupby(['unique_id','zone', 'band'], as_index=False).nth(0).index[1:]

In [108]:
g1 = test.groupby('unique_id').get_group(4)

In [109]:
# Check if group only has 1 unique zone
origin_index = g1.groupby(['unique_id','zone', 'band'], as_index=False).nth(0).index[1:]

In [72]:
origin_index

Int64Index([49, 50, 51, 52, 53, 54, 55, 56], dtype='int64')

In [110]:
inverse = g1.loc[origin_index - 1].reset_index()[['latitude','longitude']] - (g1.loc[origin_index].reset_index()[['latitude','longitude']] - g1.loc[origin_index - 1].reset_index()[['latitude','longitude']])

In [111]:
g1

,latitude,longitude,unique_id,easting,northing,zone,band,origin_e,origin_n,delta_e,delta_n
47,72,176,4,465510.981461,7.989219e+06,60,X,0,0,0.000000,0.00000
48,72,175,4,431030.462679,7.990077e+06,60,X,0,0,-34480.518783,858.71709
49,72,173,4,568969.537321,7.990077e+06,59,X,0,0,0.000000,0.00000
50,72,167,4,568969.537321,7.990077e+06,58,X,0,0,0.000000,0.00000
51,70,167,4,576330.657410,7.767125e+06,58,W,0,0,0.000000,0.00000
52,63,167,4,601293.020583,6.987165e+06,58,V,0,0,0.000000,0.00000
53,47,167,4,652049.036945,5.207105e+06,58,T,0,0,0.000000,0.00000
54,30,167,4,692915.105182,3.320469e+06,58,R,0,0,0.000000,0.00000
55,30,155,4,692915.105182,3.320469e+06,56,R,0,0,0.000000,0.00000
56,30,140,4,403549.847433,3.319206e+06,54,R,0,0,0.000000,0.00000


In [171]:
from_latlon(-81, 180)

OutOfRangeError: latitude out of range (must be between 80 deg S and 84 deg N)

In [176]:
inverse

,latitude,longitude
0,72,177
1,72,179
2,74,167
3,77,167
4,79,167
5,64,167
6,30,179
7,30,170


In [198]:
len(inverse[(inverse['latitude'] > 84) | (inverse['latitude'] < -80) | (inverse['longitude'] > 180) | (inverse['longitude'] < -180)]) == 0

True

In [191]:
inverse[(inverse['longitude'] > 175) | (inverse['longitude'] < -180)]['longitude'].any()

0    177
1    179
6    179
Name: longitude, dtype: int64

In [113]:
inverse_results = inverse.apply(north_east, axis=1, result_type='expand')

In [205]:
inverse_results

,0,1,2,3
0,500000.000000,7.988933e+06,60,X
1,568969.537321,7.990077e+06,60,X
2,561523.087773,8.213070e+06,58,X
3,550213.526462,8.547627e+06,58,X
4,542594.134555,8.770704e+06,58,X
5,597812.110084,7.098549e+06,58,W
6,692915.105182,3.320469e+06,60,R
7,403549.847433,3.319206e+06,59,R


In [122]:
inverse_results[[0,1]].values

array([[ 500000.        , 7988932.50407181],
       [ 568969.53732149, 7990077.47259416],
       [ 561523.08777257, 8213070.39889143],
       [ 550213.52646193, 8547627.18949151],
       [ 542594.13455527, 8770703.84346667],
       [ 597812.11008361, 7098548.74969565],
       [ 692915.1051815 , 3320469.28649639],
       [ 403549.84743286, 3319206.22275526]])

In [114]:
new_origin = g1.loc[origin_index-1][['easting', 'northing']].values - inverse_results[[0,1]].values + g1.loc[origin_index-1][['easting', 'northing']].values

In [121]:
new_origin

array([[ 362060.92535702, 7991222.4411165 ],
       [ 568969.53732149, 7990077.47259416],
       [ 576415.98687041, 7767084.54629688],
       [ 602447.78835893, 6986623.15425002],
       [ 659991.90660996, 5203625.45578088],
       [ 706285.96380677, 3315661.90513585],
       [ 692915.1051815 , 3320469.28649639],
       [ 982280.36293014, 3321732.35023752]])

In [115]:
g1.loc[origin_index, ['origin_e', 'origin_n']] = new_origin - g1.loc[origin_index-1][['easting', 'northing']].values + g1.loc[origin_index-1][['delta_e', 'delta_n']].values

In [375]:
inverse_results[[0,1]].values - g1.loc[origin_index][['northing', 'easting']].values

array([[ -77528.11542994,       0.        ],
       [   3702.70294026,  223055.55079327],
       [-152661.31482087,       0.        ]])

In [374]:
inverse_results[[0,1]].values - g1.loc[origin_index-1][['northing', 'easting']].values

array([[ 19368.15741722,   -832.78624999],
       [  1856.69406067, 111533.37993163],
       [ 38143.05733119,  -1564.93782538]])

In [127]:
from_latlon(72, 167)

(568969.53732149, 7990077.472594157, 58, 'X')

In [119]:
g1

,latitude,longitude,unique_id,easting,northing,zone,band,origin_e,origin_n,delta_e,delta_n,delta_cum_n,delta_cum_e,final_delta_n,final_delta_e
47,72,176,4,465510.981461,7.989219e+06,60,X,0.000000,0.000000e+00,0.000000,0.00000,0.000000e+00,0.000000,0.000000e+00,0.000000
48,72,175,4,431030.462679,7.990077e+06,60,X,0.000000,0.000000e+00,-34480.518783,858.71709,0.000000e+00,0.000000,8.587171e+02,-34480.518783
49,72,173,4,568969.537321,7.990077e+06,59,X,-103450.056104,2.003686e+03,0.000000,0.00000,2.003686e+03,-103450.056104,2.003686e+03,-103450.056104
50,72,167,4,568969.537321,7.990077e+06,58,X,0.000000,0.000000e+00,0.000000,0.00000,2.003686e+03,-103450.056104,2.003686e+03,-103450.056104
51,70,167,4,576330.657410,7.767125e+06,58,W,7446.449549,-2.229929e+05,0.000000,0.00000,-2.209892e+05,-96003.606556,-2.209892e+05,-96003.606556
52,63,167,4,601293.020583,6.987165e+06,58,V,26117.130949,-7.805020e+05,0.000000,0.00000,-1.001491e+06,-69886.475607,-1.001491e+06,-69886.475607
53,47,167,4,652049.036945,5.207105e+06,58,T,58698.886027,-1.783539e+06,0.000000,0.00000,-2.785030e+06,-11187.589580,-2.785030e+06,-11187.589580
54,30,167,4,692915.105182,3.320469e+06,58,R,54236.926862,-1.891443e+06,0.000000,0.00000,-4.676474e+06,43049.337282,-4.676474e+06,43049.337282
55,30,155,4,692915.105182,3.320469e+06,56,R,0.000000,0.000000e+00,0.000000,0.00000,-4.676474e+06,43049.337282,-4.676474e+06,43049.337282
56,30,140,4,403549.847433,3.319206e+06,54,R,289365.257749,1.263064e+03,0.000000,0.00000,-4.675211e+06,332414.595031,-4.675211e+06,332414.595031


In [450]:
g1[['latitude', 'longitude', 'easting', 'northing', 'zone', 'band', 'final_delta_e', 'final_delta_n']].round(2)

,latitude,longitude,easting,northing,zone,band,final_delta_e,final_delta_n
0,80,176,480615.20,8881752.42,60,X,0.00,0.00
1,80,175,461235.94,8882252.17,60,X,-19379.25,499.75
2,80,174,441867.78,8883084.96,60,X,-38747.41,1332.54
3,80,173,538764.06,8882252.17,59,X,-58115.57,2165.33
4,80,172,519384.80,8881752.42,59,X,-77494.82,1665.57
5,80,171,500000.00,8881585.82,59,X,-96879.63,1498.97
6,80,170,480615.20,8881752.42,59,X,-116264.43,1665.57
7,73,170,467367.68,8100752.14,59,X,-129511.95,-779334.71
8,72,170,465510.98,7989218.76,59,X,-131368.65,-890868.09
9,71,170,463664.97,7877696.58,59,W,-133225.34,-1002401.47


In [ ]:
g1['delta_cum_n'] = g1['origin_n'].cumsum()
g1['delta_cum_e'] = g1['origin_e'].cumsum()
g1['final_delta_n'] = g1['delta_n'] + g1['delta_cum_n']
g1['final_delta_e'] = g1['delta_e'] + g1['delta_cum_e']

In [116]:
g1['delta_cum_n'] = g1['origin_n'].cumsum()

/tmp/ipykernel_374/1931479025.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  g1['delta_cum_n'] = g1['origin_n'].cumsum()


In [117]:
g1['delta_cum_e'] = g1['origin_e'].cumsum()

/tmp/ipykernel_374/3516402163.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  g1['delta_cum_e'] = g1['origin_e'].cumsum()


In [118]:
g1['final_delta_n'] = g1['delta_n'] + g1['delta_cum_n']
g1['final_delta_e'] = g1['delta_e'] + g1['delta_cum_e']

/tmp/ipykernel_374/2592712619.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  g1['final_delta_n'] = g1['delta_n'] + g1['delta_cum_n']
/tmp/ipykernel_374/2592712619.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  g1['final_delta_e'] = g1['delta_e'] + g1['delta_cum_e']


In [340]:
bruh

,origin_n,origin_e
0,-77528.115430,0.000000
1,3702.702940,223055.550793
2,-152661.314821,0.000000


In [207]:
(inverse_results[[2,3]].values == g1.loc[origin_index-1][['zone', 'band']].values)

array([[ True,  True],
       [False,  True],
       [ True,  True],
       [ True, False],
       [ True, False],
       [ True, False],
       [False,  True],
       [False,  True]])

In [206]:
0 in (inverse_results[[2,3]].values == g1.loc[origin_index-1][['zone', 'band']].values)

True

In [312]:
g1.loc[origin_index-1][['zone', 'band']].values

array([[60, 'X'],
       [59, 'X'],
       [59, 'W'],
       [58, 'W'],
       [57, 'V'],
       [56, 'V']], dtype=object)

In [232]:
test.loc[origin_index]['latitude']

3     80
9     71
13    70
14    80
17    80
23    71
27    70
28    62
32    62
36    55
Name: latitude, dtype: int64

In [220]:
test.loc[origin_index, ['northing', 'easting']] - test.loc[origin_index-1, ['northing', 'easting']]

,northing,easting
2,NaN,NaN
3,NaN,NaN
8,NaN,NaN
9,NaN,NaN
12,NaN,NaN
13,0.0,0.0
14,NaN,NaN
16,NaN,NaN
17,NaN,NaN
22,NaN,NaN


In [206]:
test

,latitude,longitude,unique_id,northing,easting,zone,band,jump
0,80,176,1,480615.196704,8.881752e+06,60,X,1
1,80,175,1,461235.942285,8.882252e+06,60,X,0
2,80,174,1,441867.784868,8.883085e+06,60,X,0
3,80,173,1,538764.057715,8.882252e+06,59,X,1
4,80,172,1,519384.803296,8.881752e+06,59,X,0
5,80,171,1,500000.000000,8.881586e+06,59,X,0
6,80,170,1,480615.196704,8.881752e+06,59,X,0
7,73,170,1,467367.675522,8.100752e+06,59,X,0
8,72,170,1,465510.981461,7.989219e+06,59,X,0
9,71,170,1,463664.972582,7.877697e+06,59,W,1


In [212]:
test.locorigin_index[1:])

10

In [208]:
len(test.loc[test['jump'].shift(-1)==1])

10

In [148]:
test.loc[origin_index, ['bruh']] = 2

In [176]:
origin_index

Int64Index([0, 3, 9, 13, 14, 17, 23, 27, 28, 32, 36], dtype='int64')

In [154]:
test.loc[origin_index]['northing']

KeyError: '[-1] not in index'

In [ ]:
test[['n_delta', 'e_delta']] = test[['northing', 'easting']] - test.groupby('unique_id')[['northing', 'easting']].transform('first')

In [97]:
test

,latitude,longitude,unique_id,northing,easting,zone,band
0,80,176,1,480615.196704,8.881752e+06,60,X
1,80,175,1,461235.942285,8.882252e+06,60,X
2,80,174,1,441867.784868,8.883085e+06,60,X
3,80,173,1,538764.057715,8.882252e+06,59,X
4,80,172,1,519384.803296,8.881752e+06,59,X
5,80,171,1,500000.000000,8.881586e+06,59,X
6,80,170,1,480615.196704,8.881752e+06,59,X
7,73,170,1,467367.675522,8.100752e+06,59,X
8,72,170,1,465510.981461,7.989219e+06,59,X
9,71,170,1,463664.972582,7.877697e+06,59,W


In [94]:
test.groupby('unique_id')

In [ ]:
df[['northing', 'easting']] = df[['northing', 'easting']] - df.groupby('unique_id')[['northing', 'easting']].transform('first')

In [87]:
list(from_latlon(55, 155))

[627928.1913773258, 6096620.70703952, 56, 'U']

In [62]:
list(from_latlon(70, 167))

[576330.657410433, 7767125.1718707625, 58, 'W']

In [7]:
list(from_latlon(23.34, 173))

[704474.7971961845, 2582572.760927208, 59, 'Q']

In [ ]:
# Calculate trajectory northing and easting, get zones for each point
# if new latitude band and longitude zone, discard trajectory
# if new latitude band or longitude zone:
#   find new origin:
#   a = subtract last lat/long point of previous zone from first lat/long point of new zone
#   b = UTM(a)
#   c = UTM(last lat/long point of previous zone)
#   new_origin = c + (c - b)
#   